In [2]:
import numpy as np

In [3]:
T=200
k=100
R_y=0.02 #in [0.02, 0.25, 0.5]
rho=0.75
a=1
b=1
A=1
B=1
s=5 #in [5,10,100]

In [4]:
gamma2=np.random.beta(a,b)
q=np.random.beta(A,B)
v_x=1

In [5]:
def sample_beta(k, s):
    """Sample of beta vector of dimensions 1*k

    Args:
        k (int): number of predictors
        s (int): number of non-zero elements of beta

    Returns:
        np.array: dimensions 1*k
    """
    beta=np.zeros(k)
    index_normal_distribution=np.random.choice(len(beta), size=s, replace=False)
    beta[index_normal_distribution] = np.random.normal(loc=0, scale=1, size=s)
    return beta

def sample_Z(beta):
    Z=beta
    Z[Z!=0]=1

    return Z

def compute_R2(q, k, gamma2, v_x):
    return (q*k*gamma2*v_x)/(q*k*gamma2*v_x+1)

def compute_X(T, k, rho):
    """Compute matrix of xt observations

    Args:
        T (int): number of observations
        k (int): number of predictors
        rho (float): Toeplitz correlation parameter

    Returns:
        np.array: dimensions T*k
    """
    cov_matrix=np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            cov_matrix[i,j]=rho**np.abs(i-j)
    return np.random.multivariate_normal([0]*k, cov_matrix, T)

def compute_sigma2(R_y, beta, X):
    """ Compute sigma2
    Args:
        R_y (float): pourcentage of explained variance
        beta (np.array): beta previously sampled
        X (np.array): matrix of (xt) samples

    Returns:
        float: dimensions 1*1
    """
    return (1/R_y-1)*np.mean(np.square(X @ beta))

def sample_epsilon(T, sigma2):
    """Sample epsilon_1,...,epsilon_T

    Args:
        T (int): number of observations
        sigma2 (float): sigma2 previously sampled

    Returns:
        np.array: dimensions 1*T
    """
    return np.random.normal(loc=0, scale=sigma2, size=T)

def compute_Y(X, beta, epsilon):
    return X@beta + epsilon
    

In [6]:
beta=sample_beta(k=k, s=s)
Z=sample_Z(beta=beta)
R2=compute_R2(q=q, k=k, gamma2=gamma2, v_x=v_x)
X=compute_X(T=T, k=k, rho=rho)
sigma2=compute_sigma2(R_y=R_y, beta=beta, X=X)
epsilon=sample_epsilon(T=T, sigma2=sigma2)
Y=compute_Y(X=X, beta=beta, epsilon=epsilon)

* $l=0$ dans l'énoncé donc, $U=0$: bizarre comme certaines variances dépendent de U (celle du posterior de $\phi$) et valent donc 0
* Doute sur la génération de $\beta$ et $Z$: en théorie $\beta$, suit un mélange de lois (gaussien + dirac). Or, dans l'énoncé on demande explicitement de mettre $k-s$ élements de $\beta$ à 0 et les autres composantes suivent une normale centrée réduite. De plus, $Z_j$ suit une loi de bernouilli de param_tre $q$. Or, dans ce cas, il n'y a pas de simulation aléatoire de $Z$. On regarde juste les composantes de $\beta$ non-nulles. Je trouve ça étrange.